In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
pd.options.display.max_columns=100
pd.options.display.max_rows=300
import numpy as np

### z플립4% 서치클릭 프로 데이터 불러오기 및 전처리

In [ ]:
"""데이터 불러오기"""
data=pd.read_csv('/home/ncp/workspace/nasw2/20231129_22y2h_sc_%z플립%.csv')

In [ ]:
data.head()

In [ ]:
"""전처리1"""
data.reset_index(inplace=True)
data=data.iloc[:,2:-1]  # 필요없는 열 삭제
# data.columns=list(data.iloc[0,:]) # 컬럼 설정
# data=data.iloc[1:,:]  # 첫번째 행 삭제

In [ ]:
data.head()

In [ ]:
"""전처리2"""
data.columns = [str(col).strip() for col in data.columns]   # 컬럼명 앞뒤 공백 삭제
for col in data.columns:
    data[col] = data[col].apply(lambda x: x.strip() if isinstance(x, str) else x)   # 내용 앞뒤 공백 삭제
for col in data.columns:
    data.drop(data[data[col]==col].index, inplace=True) # 컬럼명과 동일한 행 삭제
data.columns = [col.split('.')[1] for col in data.columns]  # 컬럼명에 병합되어 있는 데이터명 삭제
data['date']=pd.to_datetime(data['date'], errors='coerce')  # 날짜 형식 변경
data['count']=data['count'].apply(lambda x: int(x) if isinstance(x, str) and x.isdigit() else np.nan)   # count 변수 (object::int)

In [ ]:
data.columns

In [ ]:
data.head()

In [ ]:
print(len(data['date'].value_counts()), '일치 데이터', sep='')  # 2주치 데이터
print(format(len(data), ','),'행 데이터', sep='')

### topic1. 브랜드 검색 분석 (search_click)
#### 1.1 고객 특징 분석

In [ ]:
# 연령대 정제
data['age2'] = data['age'].apply(lambda x: '-18' if x in ['12','13-18']
                                       else '19-29' if x in ['19-24','25-29']
                                       else '30-39' if x in ['30-34','35-39']
                                       else '40-49' if x in ['40-44','45-49']
                                       else '50-')

In [ ]:
# %z플립%
data

# z플립4
data_z4 = data[data['keyword'].isin(['z플립4'])]

In [ ]:
"""절대량 비교"""
print('##z플립 절대량')
print(format(data['count'].sum(), ','))
print('------------------------------')     # 978,104 행

"""절대량 비교"""
print('##z플립4 절대량')
print(format(data_z4['count'].sum(), ','))
print('------------------------------')     # 25,361 행

In [ ]:
"""z플립 검색 남녀 비율"""
print('##z플립 검색한 남녀 비율')
print(round(data_z4.groupby(by='gender')['count'].sum()/data_z4['count'].sum(), 2))
print('------------------------------')    

In [ ]:
print(round(data.groupby(by='age2')['count'].sum()/data['count'].sum(),2))     
print(round(data_z4.groupby(by='age2')['count'].sum()/data['count'].sum(),2))   
#원형으로 나타내기
plt.rc("font", family="NanumGothicCoding")
sns.set(font="NanumGothicCoding", rc={"axes.unicode_minus":False}, style='darkgrid')
top_pie = round(data.groupby(by='age2')['count'].sum()/data['count'].sum(),2) 
top_pie.plot(kind='pie', autopct='%2.f%%', figsize=(5,5), fontsize=15, title='전체 %갤럭시z플립4% 연령대별 검색')
plt.show()

plt.rc("font", family = "NanumGothicCoding")
sns.set(font="NanumGothicCoding", rc={"axes.unicode_minus":False}, style='darkgrid')
top_pie=round(data_z4.groupby(by='age2')['count'].sum()/data_z4['count'].sum(),2)
top_pie.plot(kind='pie', autopct='%2.f%%', figsize=(5,5), fontsize=15, title='전체 갤럭시z플립4 연령대별 검색')
plt.show()

In [ ]:
"""성-연령별 절대량"""
print("z플립4 성-연령별 검색 절대량")
tmp = pd.DataFrame(round(data.groupby(by=['gender', 'age2'])['count'].sum())).reset_index()
print(tmp.pivot(index='gender', columns='age2'))
print('------------------------------')

In [ ]:
"""성-연령별 비율 (전체비율 기준)"""
print("z플립4 성-연령별 검색 비율")
tmp = pd.DataFrame(round(data.groupby(by=['gender', 'age2'])['count'].sum())).reset_index()
print(tmp.pivot(index='gender', columns='age2'))
print('------------------------------')

In [ ]:
"""국내 지역 비율"""
print(round(data.groupby(by='loc1')['count'].sum()/data['count'].sum(),2))

# 원형으로 나타내기
plt.rc("font", family = "NanumGothicCoding")
sns.set(font="NanumGothicCoding", rc={"axes.unicode_minus":False}, style='darkgrid')
top_pie=round(data.groupby(by='loc1')['count'].sum()/data['count'].sum(),2)
top_pie.plot(kind='pie', autopct='%2.f%%', figsize=(5,5), fontsize=15, title='전체 갤럭시z플립4 연령대별 검색')
plt.show()

#### 1.2 고객 행동 분석

In [ ]:
"""검색클릭영역 전체 비율"""
print("갤럭시z플립4 지역 비율")
tmp = pd.DataFrame(round(data_z4.groupby(by=['area'])['count'].sum()/data_z4['count'].sum(),2)).reset_index()
tmp = tmp.sort_values(by='count', ascending=False)
print(tmp)
print('------------------------------')

In [ ]:
"""성-연령별 검색클릭 영역 분석: 여성"""
# 성-연령별로 나눈 비율

print("#10대 여자 갤럭시z플립4 클릭영역 탑 10")
tmp = pd.DataFrame(round(data_z4[(data_z4['gender']=='f')&(data_z4['age2'].isin(['-18']))]
                         .groupby(by=['area'])['count'].sum()/
                         data_z4.loc[(data_z4['gender']=='f')&(data_z4['age2'].isin(['-18'])),'count'].sum(),2)).reset_index()
print(tmp.sort_values(by='count', ascending=False)[:10])
print('------------------------------')

print("#20대 여자 갤럭시z플립4 클릭영역 탑 10")
tmp = pd.DataFrame(round(data_z4[(data_z4['gender']=='f')&(data_z4['age2'].isin(['19-29']))]
                         .groupby(by=['area'])['count'].sum()/
                         data_z4.loc[(data_z4['gender']=='f')&(data_z4['age2'].isin(['19-29'])),'count'].sum(),2)).reset_index()
print(tmp.sort_values(by='count', ascending=False)[:10])
print('------------------------------')


print("#30대 여자 갤럭시z플립4 클릭영역 탑 10")
tmp = pd.DataFrame(round(data_z4[(data_z4['gender']=='f')&(data_z4['age2'].isin(['30-39']))]
                         .groupby(by=['area'])['count'].sum()/
                         data_z4.loc[(data_z4['gender']=='f')&(data_z4['age2'].isin(['30-39'])),'count'].sum(),2)).reset_index()
print(tmp.sort_values(by='count', ascending=False)[:10])
print('------------------------------')


print("#40대 여자 갤럭시z플립4 클릭영역 탑 10")
tmp = pd.DataFrame(round(data_z4[(data_z4['gender']=='f')&(data_z4['age2'].isin(['40-49']))]
                         .groupby(by=['area'])['count'].sum()/
                         data_z4.loc[(data_z4['gender']=='f')&(data_z4['age2'].isin(['40-49'])),'count'].sum(),2)).reset_index()
print(tmp.sort_values(by='count', ascending=False)[:10])
print('------------------------------')


print("#50대 이상 여자 갤럭시z플립4 클릭영역 탑 10")
tmp = pd.DataFrame(round(data_z4[(data_z4['gender']=='f')&(data_z4['age2'].isin(['50-']))]
                         .groupby(by=['area'])['count'].sum()/
                         data_z4.loc[(data_z4['gender']=='f')&(data_z4['age2'].isin(['50-'])),'count'].sum(),2)).reset_index()
print(tmp.sort_values(by='count', ascending=False)[:10])
print('------------------------------')


In [ ]:
"""성-연령별 검색클릭 영역 분석: 남성"""
# 성-연령별로 나눈 비율

print("#10대 남자 갤럭시z플립4 클릭영역 탑 10")
tmp = pd.DataFrame(round(data_z4[(data_z4['gender']=='m')&(data_z4['age2'].isin(['-18']))]
                         .groupby(by=['area'])['count'].sum()/
                         data_z4.loc[(data_z4['gender']=='m')&(data_z4['age2'].isin(['-18'])),'count'].sum(),2)).reset_index()
print(tmp.sort_values(by='count', ascending=False)[:10])
print('------------------------------')

print("#20대 남자 갤럭시z플립4 클릭영역 탑 10")
tmp = pd.DataFrame(round(data_z4[(data_z4['gender']=='m')&(data_z4['age2'].isin(['19-29']))]
                         .groupby(by=['area'])['count'].sum()/
                         data_z4.loc[(data_z4['gender']=='m')&(data_z4['age2'].isin(['19-29'])),'count'].sum(),2)).reset_index()
print(tmp.sort_values(by='count', ascending=False)[:10])
print('------------------------------')


print("#30대 남자 갤럭시z플립4 클릭영역 탑 10")
tmp = pd.DataFrame(round(data_z4[(data_z4['gender']=='m')&(data_z4['age2'].isin(['30-39']))]
                         .groupby(by=['area'])['count'].sum()/
                         data_z4.loc[(data_z4['gender']=='m')&(data_z4['age2'].isin(['30-39'])),'count'].sum(),2)).reset_index()
print(tmp.sort_values(by='count', ascending=False)[:10])
print('------------------------------')


print("#40대 남자 갤럭시z플립4 클릭영역 탑 10")
tmp = pd.DataFrame(round(data_z4[(data_z4['gender']=='m')&(data_z4['age2'].isin(['40-49']))]
                         .groupby(by=['area'])['count'].sum()/
                         data_z4.loc[(data_z4['gender']=='m')&(data_z4['age2'].isin(['40-49'])),'count'].sum(),2)).reset_index()
print(tmp.sort_values(by='count', ascending=False)[:10])
print('------------------------------')


print("#50대 이상 남자 갤럭시z플립4 클릭영역 탑 10")
tmp = pd.DataFrame(round(data_z4[(data_z4['gender']=='m')&(data_z4['age2'].isin(['50-']))]
                         .groupby(by=['area'])['count'].sum()/
                         data_z4.loc[(data_z4['gender']=='m')&(data_z4['age2'].isin(['50-'])),'count'].sum(),2)).reset_index()
print(tmp.sort_values(by='count', ascending=False)[:10])
print('------------------------------')


#### 1.3 제품 검색량 추이

In [ ]:
""""전체 검색량 추이"""
fig, ax=plt.subplots(figsize=(10,10))

line, = ax.plot(data.groupby(by=['date'])['count'].sum())

In [ ]:
""""z플립4 검색량 추이"""
fig, ax=plt.subplots(figsize=(10,10))

line, = ax.plot(data_z4.groupby(by=['date'])['count'].sum())

In [ ]:
"""Z플립4 일별 검색량 합계"""
result = data_z4.groupby(by=['date'])['count'].sum()
result

#### 1.4 연관 검색어 분석

In [ ]:
"""z플립4 연관 검색어 분석"""
temp = data.groupby(by=['data','keyword'])['count'].sum().reset_index()

# keyword1, keyword2의 모든 조합에 대한 count를 sum
temp.drop(temp[temp['count']<=10].index, inplace=True)      # 용량 다운사이징: 10번 이하의 조합은 삭제
temp = temp[temp['keyword'].str.contains('z플립4')]         # view 'z플립4' related only
temp = temp.sort_values(by=['date','count'], ascending=[True,False])

# temp.to_csv('/home/ncp/workspace/nasw2/22y2h_sc_z플립4_연관검색어.csv', encoding='utf-8-sig')

temp.head()

#### 1.5 고객 니즈 분석
- note: TF 서버 내 워드클라우드 패키지 인스톨되어 있지 않아서 사용x

In [ ]:
"""전체 데이터 %z플립%에서 검색한 키워드"""
pd.DataFrame(data['keyword'].value_counts()).head(200)

In [ ]:
"""10대 여성이 검색한 키워드 목록"""
print(pd.DataFrame(data.loc[(data['age2'].isin(['-18'])) & (data['gender']=='f'), 'keyword'].value_counts()).head(200))

In [ ]:
"""20대 여성이 검색한 키워드 목록"""
print(pd.DataFrame(data.loc[(data['age2'].isin(['19-29'])) & (data['gender']=='f'), 'keyword'].value_counts()).head(200))

In [ ]:
"""30대 여성이 검색한 키워드 목록"""
print(pd.DataFrame(data.loc[(data['age2'].isin(['30-39'])) & (data['gender']=='f'), 'keyword'].value_counts()).head(200))

In [ ]:
"""40대 여성이 검색한 키워드 목록"""
print(pd.DataFrame(data.loc[(data['age2'].isin(['40-49'])) & (data['gender']=='f'), 'keyword'].value_counts()).head(200))

In [ ]:
"""50대 이상 여성이 검색한 키워드 목록"""
print(pd.DataFrame(data.loc[(data['age2'].isin(['50-'])) & (data['gender']=='f'), 'keyword'].value_counts()).head(200))

In [ ]:
"""10대 남성이 검색한 키워드 목록"""
print(pd.DataFrame(data.loc[(data['age2'].isin(['-18'])) & (data['gender']=='m'), 'keyword'].value_counts()).head(200))

In [ ]:
"""20대 남성이 검색한 키워드 목록"""
print(pd.DataFrame(data.loc[(data['age2'].isin(['19-29'])) & (data['gender']=='m'), 'keyword'].value_counts()).head(200))

In [ ]:
"""30대 남성이 검색한 키워드 목록"""
print(pd.DataFrame(data.loc[(data['age2'].isin(['30-39'])) & (data['gender']=='m'), 'keyword'].value_counts()).head(200))

In [ ]:
"""40대 남성이 검색한 키워드 목록"""
print(pd.DataFrame(data.loc[(data['age2'].isin(['40-49'])) & (data['gender']=='m'), 'keyword'].value_counts()).head(200))

In [ ]:
"""50대 이상 남성이 검색한 키워드 목록"""
print(pd.DataFrame(data.loc[(data['age2'].isin(['50-'])) & (data['gender']=='m'), 'keyword'].value_counts()).head(200))

In [ ]:
"""z플립4 성-연령별 검색 절대값"""
tmp123 = pd.DataFrame(round(data.groupby(by=['area','gender','age2'])['count'].sum())).reset_index()    # 성-연령별로 나눈 비율
tmp123.pivot(index=['area','gender'], columns = 'age2')